## Instalación de dependencias

In [3]:
# Instala memory-profiler en Colab
%pip install memory-profiler

# Necesario para mostrar el gráfico de uso de memoria en Colab
%pip install matplotlib

%pip install memory_profiler

%pip install emoji

# Preparación del archivo como antes
%pip install gdown
url = 'https://drive.google.com/uc?id=1ig2ngoXFTxP5Pa8muXo02mDTFexZzsis'
!gdown "$url" -O "archivo.zip"
!unzip -o "archivo.zip" -d "./"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 3.0 MB/s eta 0:00:00
Downloading...
From (original): https://drive.google.com/uc?id=1ig2ngoXFTxP5Pa8muXo02mDTFexZzsis
From (redirected): https://drive.google.com/uc?id=1ig2ngoXFTxP5Pa8muXo02mDTFexZzsis&confirm=t&uuid=c35fe96b-d3c2-47b7-9180-6db6daeb9503
To: /content/archivo.zip
100% 60.4M/60.4M [00:00<00:00, 175MB/s]
Archive:  archivo.zip
  inflating: ./farmers-protest-tweets-2021-2-4.json  


## Implementación de la función Q1 Time

In [15]:
import emoji
import json
from collections import Counter
from typing import List, Tuple

# Función Map: Procesa cada línea (tweet) para encontrar emojis
def map_function(line):
    try:
        tweet = json.loads(line)
        emojis_found = emoji.emoji_list(tweet['content'])
        return [item['emoji'] for item in emojis_found]
    except json.JSONDecodeError:
        return []

# Función Shuffle: Organiza los emojis encontrados para su procesamiento en el Reduce
def shuffle_and_sort(mapped_values):
    all_emojis = []
    for emoji_list in mapped_values:
        all_emojis.extend(emoji_list)
    return Counter(all_emojis)

# Función Reduce: Agrega los conteos de cada emoji
def reduce_function(shuffled_data):
    return shuffled_data.most_common(10)

def q2_time(file_path: str) -> List[Tuple[str, int]]:
    mapped_values = []

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            mapped_values.extend(map_function(line))

    shuffled_data = shuffle_and_sort(mapped_values)
    reduced_data = reduce_function(shuffled_data)

    # Ya no escribe en un archivo, sino que retorna los datos directamente
    return reduced_data

# Llamar a la función principal para ejecutar el MapReduce y obtener el resultado
file_path = './farmers-protest-tweets-2021-2-4.json'
result = q2_time(file_path)
print(result)

[('🙏', 7286), ('😂', 3072), ('️', 2985), ('🚜', 2972), ('✊', 2411), ('🌾', 2363), ('🇮', 2096), ('🇳', 2094), ('🏻', 2080), ('❤', 1779)]


# Medición del consumo de memoria y tiempo de ejecución de la función Q1 Time


Resultado de medición expuesto por medio de un archivo jpg

In [14]:
import json
from collections import defaultdict, Counter
from datetime import datetime
import cProfile, pstats
from memory_profiler import memory_usage
import matplotlib.pyplot as plt
import emoji
import json
from collections import Counter
from typing import List, Tuple

# Función Map: Procesa cada línea (tweet) para encontrar emojis
def map_function(line):
    try:
        tweet = json.loads(line)
        emojis_found = emoji.emoji_list(tweet['content'])
        return [item['emoji'] for item in emojis_found]
    except json.JSONDecodeError:
        return []

# Función Shuffle: Organiza los emojis encontrados para su procesamiento en el Reduce
def shuffle_and_sort(mapped_values):
    all_emojis = []
    for emoji_list in mapped_values:
        all_emojis.extend(emoji_list)
    return Counter(all_emojis)

# Función Reduce: Agrega los conteos de cada emoji
def reduce_function(shuffled_data):
    return shuffled_data.most_common(10)

def q2_time(file_path: str) -> List[Tuple[str, int]]:
    mapped_values = []

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            mapped_values.extend(map_function(line))

    shuffled_data = shuffle_and_sort(mapped_values)
    reduced_data = reduce_function(shuffled_data)

    # Ya no escribe en un archivo, sino que retorna los datos directamente
    return reduced_data


def profile_memory_and_cpu():
    archivo_json = './farmers-protest-tweets-2021-2-4.json'

    # Profile de memoria
    mem_usage = memory_usage((q2_time, (archivo_json,)), interval=0.1)

    # Profile de tiempo de CPU
    profiler = cProfile.Profile()
    profiler.enable()
    q2_time(archivo_json)
    profiler.disable()
    stats = pstats.Stats(profiler).sort_stats('cumtime')
    stats.print_stats()

    # Plot de uso de memoria y guardar en archivo .jpg
    plt.figure(figsize=(10, 6))
    plt.plot(mem_usage)
    plt.title('Uso de memoria a lo largo del tiempo')
    plt.xlabel('Tiempo (en intervalos de 0.1s)')
    plt.ylabel('Memoria usada (MB)')
    plt.savefig('q2_memory_resources_map_reduce.jpg')
    plt.close()

profile_memory_and_cpu()

         87415255 function calls in 34.818 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    1.561    1.561   34.818   34.818 <ipython-input-14-d02ba7bfd7b3>:32(q2_time)
   117407    0.204    0.000   33.073    0.000 <ipython-input-14-d02ba7bfd7b3>:13(map_function)
   117407    0.094    0.000   29.514    0.000 /usr/local/lib/python3.10/dist-packages/emoji/core.py:283(emoji_list)
   117407    6.952    0.000   29.419    0.000 /usr/local/lib/python3.10/dist-packages/emoji/core.py:290(<listcomp>)
 17140706   14.149    0.000   20.902    0.000 /usr/local/lib/python3.10/dist-packages/emoji/tokenizer.py:158(tokenize)
 17023302    3.225    0.000    5.522    0.000 <string>:1(<lambda>)
   117407    0.156    0.000    3.330    0.000 /usr/lib/python3.10/json/__init__.py:299(loads)
   117407    0.238    0.000    3.122    0.000 /usr/lib/python3.10/json/decoder.py:332(decode)
   117407    2.709    0.000    2.709    0.000 /usr/li